In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  
from IPython.display import HTML

In [2]:
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, precision_score, recall_score
import pandas
import numpy as np
import json
import matplotlib.pyplot as plt
import os

pandas.options.display.float_format = '{:,.3f}'.format

In [3]:
from evaluate import load, get_results, get_labels, transform_binary_probabilities, transform_binary_predictions, calculate_accuracy_loss, plot_confusion_matrix, plot_precision_recall, plot_roc_curve, calculate_pr_auc, calculate_confusion_matrix_stats, calculate_confusion_matrix, plot_tsne, accession_from_filename, plot_expert_confusion, plot_grad_cam

Using TensorFlow backend.


In [4]:
from data import describe, all_features, data, print_describe
from run import characterize_data

# All lesions

In [5]:
DIRECTORY = "c3-c4-free/data"
FEATURES = "{}/features.csv".format(DIRECTORY)
RAW = "{}/raw".format(DIRECTORY)
TRAIN = "{}/train".format(DIRECTORY)
VALIDATION = "{}/validation".format(DIRECTORY)
MODEL = "/data/intrepidlemon/deep-ultrasound/output2/models/303abda9-4663-4705-88f7-1046bbbe69fa-v2.h5"

## Lesion descriptions

In [6]:
print_describe(raw=RAW, features=FEATURES)

category: C3
	count: 131
	number lesions: 198
	column: feat
		 {'benign': 113, 'malignant': 18}
	column: imag
		 {'biopsy': 6, 'mri': 125}
category: C4
	count: 44
	number lesions: 59
	column: feat
		 {'benign': 10, 'malignant': 34}
	column: imag
		 {'biopsy': 16, 'mri': 28}


In [7]:
train, validation, _ = data(shuffle_train=False, validation_dir=VALIDATION, train_dir=TRAIN)

Found 315 images belonging to 2 classes.
Found 315 images belonging to 2 classes.
Found 245 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [8]:
print("training data breakdown: ", characterize_data(train))
print("validation data breakdown: ", characterize_data(validation))

training data breakdown:  {'benign': 216, 'malignant': 99}
validation data breakdown:  {'benign': 171, 'malignant': 74}


In [ ]:
model = load(MODEL)

In [ ]:
results = get_results(model, validation)

In [ ]:
probabilities = transform_binary_probabilities(results)
predictions = transform_binary_predictions(results)
labels = get_labels(validation)

## Accuracy and loss

In [ ]:
loss, accuracy = calculate_accuracy_loss(model, validation)
print("Accuracy:", accuracy)
print("Loss:", loss)

## Confusion matrix

In [ ]:
plot_confusion_matrix(validation, results)

In [ ]:
stats = calculate_confusion_matrix_stats(validation, results)
pandas.DataFrame(stats, index=validation.class_indices)

## ROC Curve standalone

In [ ]:
plot_roc_curve(validation, results)

In [ ]:
print("ROC AUC:", roc_auc_score(labels, probabilities))

## Precision Recall

In [ ]:
print("Average precision:", average_precision_score(labels, probabilities))
print("Precision:", precision_score(labels, predictions), "Recall:", recall_score(labels, predictions))

In [ ]:
plot_precision_recall(validation, results)

In [ ]:
print("PR AUC:", calculate_pr_auc(validation, results))

## F1 Score

In [ ]:
print("F1 score:", f1_score(labels, predictions))

## TSNE

In [ ]:
malignancy, modality, category = all_features(features=FEATURES)

In [ ]:
tsne_dataset = train

In [ ]:
tsne_labels = [category.get(accession_from_filename(f), "unknown") for f in tsne_dataset.filenames]

In [ ]:
plot_tsne(model, "dense_5", tsne_dataset, tsne_labels, perplexity=5)

## Experts

In [ ]:
expert_1 = plot_expert_confusion("evaluations/radiology-9809.json", validation)

In [ ]:
pandas.DataFrame(expert_1, index=validation.class_indices)

## ROC with experts

In [ ]:
plot_roc_curve(validation, results, experts=[{ **expert_1, "name":"expert 1" }, { **expert_1, "name":"expert 2" }])

## Grad Cam

### benign example

In [ ]:
plot_grad_cam(
    os.path.join(validation.directory, validation.filenames[0]), 
    model, 
    "dense_6", 
)

### malignant example

In [ ]:
plot_grad_cam(
    os.path.join(validation.directory, validation.filenames[len(validation.filenames) - 1]), 
    model, 
    "dense_6", 
)

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')